In [4]:
from PyInventor import *

def makeHoles(filename):
    input = open(filename, 'r')
    read = input.readlines()
    coordinates = []

    for line in read:
        coordinates.append(line.strip().split(', '))

    print("Making Holes From: '"+str(filename)+"' ...")

    
    '''
    The following code creates a new 3D Multimode Cavity
    '''
    #closes all of the open parts
    com_obj().close_all_parts()

    #set document units
    units='imperial'

    #set it to overwrite file every time the part is instantiated (REMEMBER TO PUT SAVE AT THE BOTTOM!)
    overwrite=True

    #filename
    fname='3DMM Demo.ipt'

    #path
    path='C:\\Users\\Public\\Documents\\Demos'

    #setup part 
    part=iPart(path=path, prefix=fname, units=units, overwrite=overwrite)

    #set view as shaded with edges
    part.set_visual_style(shaded=True, edges=True, hidden_edges=False)

    #set some basic stock parameters (taken from line 1 of the file)
    stock_width = float(coordinates[0][0])
    stock_length = float(coordinates[0][1])

    #set stock height
    H_stock=2.25

    '''
    IMPORTANT: 
    The typical convention is for the origin to be (0,0) but instead of this being the bottom left corner
    of the cavity, Inventor adopts the convention of making the bottom right the origin. To avoid this, the origin has
    been set to (stock_width, 0) which shifts the point back to the bottom left corner.
    '''

    #set origin
    stock_origin=(stock_width,0) 

    #set starting workplane
    lower_wp=part.add_workplane(plane='xy')

    #attach a sketch to the workplane
    sketch_1=part.new_sketch(lower_wp)

    #set upper workplane
    # upper_wp=part.add_workplane(plane='xy', offset=H_stock)
    upper_wp=part.add_workplane(plane='xy', offset=H_stock)

    #attach a sketch to the upper workplane
    sketch_2=part.new_sketch(upper_wp)

    #create stock base associated with sketch 1. The structure object keeps track of position
    stock_base=structure(part, sketch_1, start=stock_origin)
    stock_base.add_line(stock_width, 180)
    stock_base.add_line(stock_length, 90)
    stock_base.add_line(stock_width, 0)

    #structure identifier
    base_shape=stock_base.draw_path(close_path=True)

    #extrude base shape. It is important to set the object collection (obj_collection) to the specific polygon shape we created
    part.extrude(sketch_1, thickness=H_stock, obj_collection=base_shape, direction='positive', operation='join')

    #fits whole cavity in frame
    part.view.Fit()

    #create new structure representing bottom holes attached to bottom workplane
    cav=structure(part, sketch_2)

    '''
    makeHole() creates a new hole in the cavity with a default diameter of 0.1875 in and depth of 0.5 in
    '''
    
    def makeHole(x, y, dia = 0.1875, depth = 0.5):
        part.new_hole(cav.sketch, (x, y), dia, depth, direction='positive', FlatBottom=False, BottomTipAngle=None)

    for point in coordinates[1:]:
        makeHole(float(point[0]), float(point[1]))

    print(" -- Holes Made Succesfully -- ")

In [5]:
filename = "design.txt"
makeHoles("./HoleMaker/Designs/"+filename)

Making Holes From: './HoleMaker/Designs/design.txt' ...
 -- Holes Made Succesfully -- 
